# `aws lambda`

think of the steps we needed to go through before we were able to execute our sad `alarm_clock.py` script in the previous lectures and homeworks:

1. provision an `ec2` instance
2. make an `ssh` connection to that server
3. copy our `python` script to that server
4. create an environment capable of executing that `python` script
5. establish an `iam role` for `aws cli` and `boto3` process running on that server
6. add a `crontab` entry to run that script with some frequency
7. leave our `ec2` server up and running while the script runs once a day

that's a fair amount of work and infrastructure for doing something as simple as running a block of code. 

it's also relatively expensive: we didn't *need* our `ec2` service to be up and running constantly -- only for a few seconds each day. And we didn't utilize all of the resources available to our machine while we were executing that code, so even that time was somewhat wasted.

`aws lambda` is a serverless code execution service which tries to provide a more streamlined and simple process for executing simple code in a way that is highly integrated with other `aws` services.

it also generalizes from *scheduling* to *triggered* events -- meaning we can execute code with some temporal frequency, *or* any time some particular thing happens (*e.g.* a file is posted to a particular `s3` bucket). this makes it a sort of "if this then that" for `aws`

## how `lambda` functions are defined

a `lambda` function is comprised of three things:

1. a runtime and an environment: a particular language and particular versions of that language, other environment setups
2. a trigger: some event that causes a function to run
3. a function: some code executed in that run-time after that trigger

### run-time and execution environment

the available runtimes are updated with some frequency, so it's usually best to look it up and check out [this page](https://docs.aws.amazon.com/lambda/latest/dg/current-supported-versions.html). of course, the `lambda` function creation dialog will also make you choose one of them.

at the time of writing, though, the environment and runtimes were as follows:

+ os: all executed on the Amazon Linux AMI, 64 bit
+ runtimes:
    + `node.js` v4.3.2 and 5.10.3
    + `java` 8
    + **`python` 3.6 and 2.7**
    + `C# .NET Core` 1.0.1
+ sdks
    + `javascript`
    + `boto3` for `python 2.7, python 3.6`

### triggers

there is a *huge* list of events and items that could trigger the execution of your `lambda` function. for example,

1. any event in any `aws` application
2. an open `api`
3. an alexa
4. nothing at all (manual)

moral of the story: you can *probably* trigger it the way you want. It may not be obvious, but I bet you can do it.

### functions

at their heart, every `aws lambda` function is a zip directory of code that can be executed in the given, fixed runtime from above, and a single functional endpoint called a "handler". 

in the `python` world, that endpoint is specified as a python module (basically, a file name) and single file in that file which looks like

```python
# file: file_name.py
def function_name(event, context):
    # does something using the event json and the context
```

the handler here is `file_name.function_name`

#### `event`

the `event` item is a `json` object that is usually completely defined by the triggering event you chose above. 

this means that you need to know the structure of that event `json` object if you wish to perform any function logic with it, or not care about it and just resond to the fact that *some* event happened.

either way, though, it is expected that your `lambda` function code will take the event in as its first parameter

##### example `event`s

TBD

#### `context`

in each of the runtimes, `aws` creates an object called a `context` that allows the user to interactively interact with the runtime and environment. the [`python` context object is discussed here](http://docs.aws.amazon.com/lambda/latest/dg/python-context-object.html).

it will tell you (among many other things):

+ how long till your function times out
+ the memory limit of this `lambda` function
+ the function name or version

use of the `context` is pretty... context dependent. use of the `event` is much more common, though, so it's good to get used to examining the event structure

<div align="center">**walkthrough: create a hello world `lambda` function**</div>

this is following [the walkthrough in the `lambda` documentation](https://docs.aws.amazon.com/lambda/latest/dg/getting-started-create-function.html)

1. go to [the `lambda` service dashboard](https://console.aws.amazon.com/lambda/home?region=us-east-1#/home)
2. create a function
    1. blueprint
        1. search for hello-world-python3 and select it
    2. triggers
        1. do nothing and click "next"
    3. function
        1. give it whatever name you want
        2. the rest of the configurations are okay
    4. review
        1. just do it!
3. testing it
    1. be on the function page
        1. you were dropped there after creating it
        2. but if you left, you can get there be clicking "functions" in the left panel and selecting the name of your function
    2. click the "test" button
    3. use the default `json`
        1. if there is no default, `{"key3": "value, "key2": "value2",, "key1": "value1"}`
4. monitoring
    1. click the monitoring tab
        1. check out the invocation and time used graph
    2. click on the cloudwatch logs
        1. look at the log stream 

## `trigger`s worth knowing about

there are a lot of options for triggers, and depending on what you want to do with `aws` you will explore any number of them. that being said, there are a few that have immediate impact on services we've used already or will be likely to use while doing data science.

the big question to ask is:

*what does the `json` object passed from the triggered event look like, and how can / should we use it in our function?*

### scheduled events

the `lambda` world has its own version of `cron` in `CloudWatch Events`. the same way that we were able to create shell scripts that could be executed on some frequency, we can creaet `lambda` functions and execute them with some frequency.

when we set the frequency of the scheduled event, we can use `cron` syntax, or rate expressions -- docs are [here](https://docs.aws.amazon.com/lambda/latest/dg/tutorial-scheduled-events-schedule-expressions.html)

```bash
# rate example: daily
rate(1 day)

# cron example: work weekdays at 6 AM UTC
cron(0 6 * * MON-FRI *)
```

whichever we pick, an actual `cloudwatch` event `rule` is created. we could have created the rule ahead of time via the `cloudwatch` service, or we could let `lambda` build one for us.

I've already created a simple `lambda` function and a simple rule, so let's check out [the `cloudwatch` service](https://console.aws.amazon.com/cloudwatch/home?region=us-east-1) to see an example rule to get some context

<div align="center">**walkthrough: looking at `cloudwatch` scheduled event options**</div>

the walkthrough goes through the following pages

+ navigate to the `cloudwatch` service
+ select the "Events > Rule" page from the left menu
+ select any rule to go to the rule's summary page
+ click edit
    + source
        + event pattern (trigger on *any* event across `aws` services)
        + schedule (rate or cron expression
        + dropdown of sample event indicates exactly what the event is
    + target: `lambda` function
        + this happens to point at a `lambda` function (because it was built for us via `lambda`)
        + the `input` discussed here is the `input` `event` object for the `lambda` function
        + we have options
            + the exact object
            + a single sub-element from the object
            + a *constant* version
            + a value constructure from the components of the original event object (based on key-value replacement)

so, the basic `event` object is

```json
// cloudwatch scheduled event example
{
  "account": "123456789012",
  "region": "us-east-1",
  "detail": {},
  "detail-type": "Scheduled Event",
  "source": "aws.events",
  "time": "1970-01-01T00:00:00Z",
  "id": "cdc73f9d-aea9-11e3-9d5a-835b769c0d9c",
  "resources": [
    "arn:aws:events:us-east-1:123456789012:rule/my-schedule"
  ]
}
```

we could take our `hello world` example and make a small modification to incorporate elements of this event:

```python
def lambda_handler(event, context):
    print("event = {}".format(event))
    return 'hello world! it is {}'.format(event['time'])
```

<div align="center">**mini exercise: create a scheduled hello world `lambda` function**</div>

1. create a new `lambda` function with whatever name you'd like
2. trigger the event on a `cloudwatch` scheduled event
3. schedule the even to trigger every M-F at 6:00 GMT
4. use the definition below for the function
5. test your lambda function

```python
def lambda_handler(event, context):
    print("event = {}".format(event))
    return 'hello world! it is {}'.format(event['time'])
```

### `s3`

### api gateway

this service allows you to create your own RESTful api endpoint -- a url to which you may send `GET, POST, PUT,` or `DELETE` requests to perform some action. that action that you perform can very easily be a `lambda` function!

+ we made one, code is below
+ we haven't invoked one yet, need to do that

### `kinesis` and `firehose`

### `rds`

+ Lambda
    + Makeup of lambda object:
        + A function
    + free tier limits
        + https://aws.amazon.com/lambda/pricing/
    + Exercise: "the world’s worst alarm clock": 
        + Goal: create a job to post a file to S3 saying "WAKE UP" every day at 7 AM
        + Creating the function
            + Nav to "AWS Lambda" page
            + "Get started Now"
            + Examine blueprints, but "Author from scratch"
            + Ignore the trigger (for now)
            + Choose descriptive name
            + Runtime == python 3.6
            + "Edit code inline" and add the bad alarm clock python file from the s3 example
            + Create new role from template, name it something simple like "lambda_basic_execution"
            + Finish
        + Testing with our own test event
            + Test json object:
                + `{"message": "hello world", "bucket": "YOUR BUCKET NAME HERE"}`
            + Did it work?
            + Why might access be denied?
        + Permissions
            + Go to the bucket permissions
            + Also go to the IAM console (more on that below)
            + Go to policy generator
            + Figure out how to update the policy
            + Answer:
            + Add the block
                {
                    "Sid": "LambdaPutObject",
                    "Action": "s3:PutObject",
                    "Effect": "Allow",
                    "Principal": {
                        "AWS": [
                            "arn:aws:iam::336188965589:role/service-role/gu511_lambda_basic_execution"
                        ]
                    },
                    "Resource": "arn:aws:s3:::gu511.lamberty.io/*"
                }
            + See the policy for my bucket if necessary
            + Test again
        + Adding a timed trigger and specifying the event
            + "Cloudwatch event"
            + Create a cron rule
                + Set the time to 7 AM EST (note: must convert that time to GMT), and enter it in as a cron rule
            + Update the target "input" (the thing we send to be input to the target lambda function) to be {"message": "YOUR ALARM + CLOCK MESSAGE", "bucket": "YOUR BUCKET NAME HERE"}
        + Check out the logs for this rule in CloudWatch (this is the case for *all* lambda functions, independent of whether or not the *triggering* event was a lambda function)
    + Exercise: compare and contrast the cron setup to the lambda setup
        + What were the pros of each? What were the cons?
+ Can you think of situations in which you might want to use one method instead of the other?

```python
# simple hello world version
def lambda_handler(event, context):
    print("event = {}".format(event))
    return event['key1']
```

```python
# api gateway lambda function
import boto3
import json

print('Loading function')

def respond(err, res=None):
    return {
        'statusCode': '400' if err else '200',
        'body': err.message if err else json.dumps(res),
        'headers': {
            'Content-Type': 'application/json',
        },
    }


def lambda_handler(event, context):
    '''Demonstrates a simple HTTP endpoint using API Gateway. You have full
    access to the request and response payload, including headers and
    status code.

    To scan a DynamoDB table, make a GET request with the TableName as a
    query string parameter. To put, update, or delete an item, make a POST,
    PUT, or DELETE request respectively, passing in the payload to the
    DynamoDB API as a JSON body.
    '''
    print("Received event: " + json.dumps(event, indent=2))

    reqtype = event['httpMethod']
    if reqtype == 'GET':
        return respond(
            err=None, 
            res='value 1 is {}'.format(event['queryStringParameters'].get('key1'))
        )
    else:
        return respond(ValueError('Unsupported method "{}"'.format(reqtype)))
```